In [1]:
from api_keys import hf_api
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]=hf_api

In [33]:
from huggingface_hub import InferenceClient
model = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.1")
question = "I would love to open an Indian Restaurant. Suggest a fancy and attractive name for it.Do not include any extra text, just a single name"
response = client.text_generation(question, max_new_tokens=10, temperature=0.7)
print(response.strip())

.

“Spice Haven”


In [6]:
import os
from langchain_google_genai import GoogleGenerativeAI
from api_keys import gemini_api

# Set your Gemini API key
os.environ["GOOGLE_API_KEY"] = gemini_api

# Load Gemini model
llm = GoogleGenerativeAI(model="gemini-1.5-pro")

# Ask for a restaurant name
question = "I would love to open an italian Restaurant. Suggest a fancy and attractive name for it.Do not include any extra text, just a single name"
name = llm.invoke(question)

# Print only the name
print(name.strip())


Belladonna


In [24]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables = ['cusine'],
    template = "I would love to open an {cuisine} Restaurant. Suggest a fancy and attractive name for it, just a name"
)
prompt_template_name.format(cuisine = "Mexican")

'I would love to open an Mexican Restaurant. Suggest a fancy and attractive name for it, just a name'

In [26]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template_name)
chain.run("Indian")

'Saffron & Silk'

In [29]:
# Simple Sequential Chain 

llm_model = GoogleGenerativeAI(model = "gemini-1.5-pro", temperature=0.7)
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template = "I would love to open an {cuisine} Restaurant. Suggest a fancy and attractive name for it, just a name"
)
name_chain = LLMChain(llm = llm_model, prompt = prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as comma seperated values"
)
food_items_chain = LLMChain(llm = llm_model, prompt = prompt_template_items)

In [33]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain,food_items_chain])
response=chain.run("India")
print(response)

Dal Makhani, Butter Chicken, Palak Paneer, Rogan Josh, Tandoori Chicken, Samosas, Vegetable Biryani, Lamb Korma, Naan, Garlic Naan, Aloo Gobi, Chicken Tikka Masala, Gulab Jamun, Mango Lassi, Masala Chai


In [35]:
# Here, the Sequential Chain is only printing one output
# i.e, it is taking the restaurant name as input and printing the menu items 
# it is not printing the initial restaurant name
# so, to print that, we'll use Sequential chain, not the SimpleSequentialChain

In [37]:
# Sequential Chain

llm_model = GoogleGenerativeAI(model = "gemini-1.5-pro", temperature = 0.7)
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I would love to open an {cuisine} Restaurant. Suggest a fancy and attractive name for it, just a name"
)
name_chain = LLMChain(llm = llm_model, prompt=prompt_template_name, output_key = "restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as comma seperated values"
)
items_chain=LLMChain(llm = llm_model,prompt=prompt_template_items,output_key="menu_items")

In [39]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains=[name_chain,items_chain],
    input_variables = ['cuisine'],
    output_variables=['restaurant_name','menu_items']
)
chain({'cuisine':'Arabic'})
#Here, we don't use the .run fuction, we directly pass the argument, since, there could be any possible cuisines, we use a disctionary for arguments

C:\Users\Maniram\AppData\Local\Temp\ipykernel_3196\193685649.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'cuisine':'Arabic'})


{'cuisine': 'Arabic',
 'restaurant_name': 'Alhambra',
 'menu_items': 'Tapas: Patatas Bravas, Gambas al Ajillo, Tortilla Española, Croquetas de Jamón, Espinacas con Garbanzos, Boquerones en Vinagre, Pimientos de Padrón.\n\nMains: Paella Valenciana, Paella de Marisco, Fideuà, Cochinillo Asado, Rabo de Toro, Solomillo al Whisky, Bacalao a la Vizcaína.\n\nDesserts: Crema Catalana, Tarta de Santiago, Churros con Chocolate, Flan de Huevo, Arroz con Leche.\n\nDrinks: Sangria, Tinto de Verano, Cerveza, Vino Tinto, Vino Blanco.\n\n\nSo, as a comma separated list:  Patatas Bravas, Gambas al Ajillo, Tortilla Española, Croquetas de Jamón, Espinacas con Garbanzos, Boquerones en Vinagre, Pimientos de Padrón, Paella Valenciana, Paella de Marisco, Fideuà, Cochinillo Asado, Rabo de Toro, Solomillo al Whisky, Bacalao a la Vizcaína, Crema Catalana, Tarta de Santiago, Churros con Chocolate, Flan de Huevo, Arroz con Leche, Sangria, Tinto de Verano, Cerveza, Vino Tinto, Vino Blanco'}